In [1]:
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# import warnings
# warnings.filterwarnings('ignore')
# warnings.warn("once")
# %matplotlib inline
# #color = sns.color_palette()

# # Graph 한글 표시, https://lsh-story.tistory.com/83
# import matplotlib
# from matplotlib import font_manager, rc
# font_name = font_manager.FontProperties(fname="./korean.ttf").get_name()
# rc('font',family='AppleGothic')
# matplotlib.rcParams['axes.unicode_minus'] = False

# print('Python version : ', sys.version)

Python version :  3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
